In [1]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}}}

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Наименования-столбцов" data-toc-modified-id="Наименования-столбцов-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Наименования столбцов</a></span></li><li><span><a href="#Проверка-данных-и-конвертирование-их-типов" data-toc-modified-id="Проверка-данных-и-конвертирование-их-типов-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Проверка данных и конвертирование их типов</a></span></li><li><span><a href="#Дубликаты" data-toc-modified-id="Дубликаты-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Дубликаты</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Без-преобразования" data-toc-modified-id="Без-преобразования-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Без преобразования</a></span></li><li><span><a href="#С-преобразованием" data-toc-modified-id="С-преобразованием-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>С преобразованием</a></span></li><li><span><a href="#Сравнение-метрик" data-toc-modified-id="Сравнение-метрик-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Сравнение метрик</a></span></li></ul></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [3]:
df = pd.read_csv('/datasets/insurance.csv')

In [4]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


### Наименования столбцов

От кириллического шрифта нужно избавиться во избежание проблем.

In [5]:
df.columns = ['gender', 'age', 'salary', 'family_members', 'insurance_payments']

In [6]:
df

,gender,age,salary,family_members,insurance_payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


### Проверка данных и конвертирование их типов

In [7]:
for column in ['age','salary']:
    print(df[column].describe(),'\n')

count    5000.000000
mean       30.952800
std         8.440807
min        18.000000
25%        24.000000
50%        30.000000
75%        37.000000
max        65.000000
Name: age, dtype: float64 

count     5000.000000
mean     39916.360000
std       9900.083569
min       5300.000000
25%      33300.000000
50%      40200.000000
75%      46600.000000
max      79000.000000
Name: salary, dtype: float64 



In [8]:
for column in ['age','salary']:
    df[column] = df[column].astype('int')

In [9]:
for column in ['gender', 'family_members', 'insurance_payments']:
    print(df[column].value_counts(),'\n')

0    2505
1    2495
Name: gender, dtype: int64 

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: family_members, dtype: int64 

0    4436
1     423
2     115
3      18
4       7
5       1
Name: insurance_payments, dtype: int64 



Аномалий нет.

### Дубликаты

In [10]:
df.duplicated().value_counts()

False    4847
True      153
dtype: int64

Учитывая количество вводных и то, что зарплаты и возраста имеют существенный разброс, такое количество повторов не может быть случайностью.<br>
Во избежание удаления информации, стоило бы использовать уникальные идентификаторы клиентов.

In [11]:
df = df.drop_duplicates().reset_index(drop=True)

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество предсказаний не изменится.

**Обоснование:**

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w = (X^T X)^{-1} X^T y
$$
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
....
$$
$$
.....
$$
$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$
$$
w' = P^{-1}w
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
.......
$$

$$
w' = P^{-1} (X^T X)^{-1} E X^T y
$$

$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
$$
w = (X^T X)^{-1} X^T y
$$
$$
w' = P^{-1}w
$$

Предсказания:
$$
a = X*w
$$
$$
a' = X*P*w' = X*P*P^{-1}w = X*E*w = X*w
$$

$$
a' = a
$$

## Алгоритм преобразования

**Алгоритм**

1. Создать случайную матрицу.
2. Проверить, что её определитель не равен нулю.
3. Умножить матрицу признаков на сгенерированную обратимую матрицу.

**Обоснование**

Полученный результат при использовании регрессии не будет отличаться, если матрица обратима.

In [12]:
def generate_matrix(in_feat):
    X = in_feat.shape[1]
    while True:
        Matrix = np.random.randint(1,99999,size = (X,X))
        if np.linalg.det(Matrix) !=0:
            break
    return Matrix

## Проверка алгоритма

In [13]:
df_train, df_valid  = train_test_split(df, test_size=1/4, random_state=10001)

model = LinearRegression()

features_train = df_train.drop('insurance_payments', axis=1)
target_train = df_train['insurance_payments']
features_valid = df_valid.drop('insurance_payments', axis=1)
target_valid = df_valid['insurance_payments']
train_index = df_train.index
valid_index = df_valid.index

### Без преобразования

In [14]:
model.fit(features_train,target_train)
prediction = model.predict(features_valid)
score = r2_score(target_valid, prediction)
score

0.37924339961703435

### С преобразованием

Для точности эксперимента повторим его 100 раз.

In [15]:
features = df.drop('insurance_payments', axis=1)

In [16]:
df_r = pd.DataFrame((features @ generate_matrix(features)))
df_r.columns = features.columns

In [17]:
score_list = []
for x in range(100):
    matrix = generate_matrix(features)

    features_train = features.iloc[train_index] @ matrix
    features_valid = features.iloc[valid_index] @ matrix

    model.fit(features_train,target_train)
    prediction = model.predict(features_valid)
    score_r = r2_score(target_valid,prediction)
    score_list.append(score_r)

In [18]:
pd.Series(score_list).describe()

count    1.000000e+02
mean     3.792434e-01
std      6.180162e-13
min      3.792434e-01
25%      3.792434e-01
50%      3.792434e-01
75%      3.792434e-01
max      3.792434e-01
dtype: float64

### Сравнение метрик

In [19]:
print(pd.Series(score_list).mean())
score

0.37924339961708214


0.37924339961703435

Метрика R2 не изменилась.